In [1]:
#!pip install opensoundscape==0.6.2
#from opensoundscape.preprocess.preprocessors import BasePreprocessor
#import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
#from opensoundscape.torch.models import cnn
#from opensoundscape.torch.architectures import cnn_architectures
from WTS_chunking import annotation_chunker_no_duplicates, extact_split, fill_no_class
import os
import shutil
#from pydub import AudioSegment

In [2]:
train_clips = pd.read_csv("C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/automated_cosmos_tweety_to_file.csv", sep=',')
test_clips = pd.read_csv('C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/cosmos_labeled_data_files_added.csv')

train_clips = train_clips.drop(columns="Unnamed: 0")

In [3]:
test_anntotations = train_clips#.sample(5)
test_anntotations

,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH
0,1.883721,0.116279,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC100027 - Southern Nightingale-Wren - Microce...,0,81.528163,44100,Microcerculus marginatus,0.543653,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
1,3.976744,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC100027 - Southern Nightingale-Wren - Microce...,0,81.528163,44100,Microcerculus marginatus,0.230806,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
2,5.976744,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC100027 - Southern Nightingale-Wren - Microce...,0,81.528163,44100,Microcerculus marginatus,0.204163,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
3,6.023256,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC100027 - Southern Nightingale-Wren - Microce...,0,81.528163,44100,Microcerculus marginatus,0.264257,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
4,6.069767,0.116279,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC100027 - Southern Nightingale-Wren - Microce...,0,81.528163,44100,Microcerculus marginatus,0.544028,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
...,...,...,...,...,...,...,...,...,...,...
55642,1.511628,0.325581,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC9881 - Blue-grey Tanager - Thraupis episcopu...,0,14.700000,44100,Thraupis episcopus,0.943661,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55643,1.860465,0.139535,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC9881 - Blue-grey Tanager - Thraupis episcopu...,0,14.700000,44100,Thraupis episcopus,0.855924,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55644,5.976744,0.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC9881 - Blue-grey Tanager - Thraupis episcopu...,0,14.700000,44100,Thraupis episcopus,0.775321,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55645,6.627907,0.255814,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC9881 - Blue-grey Tanager - Thraupis episcopu...,0,14.700000,44100,Thraupis episcopus,0.736337,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...


In [4]:
#train_clips = annotation_chunker_no_duplicates(train_clips, 3, False)

def create_chunk_row(row, rows_to_add, new_start, new_end):
    chunk_row = row.copy()
    chunk_row["DURATION"] = new_end
    chunk_row["OFFSET"] = new_start
    rows_to_add.append(chunk_row.to_frame().T)
    return rows_to_add

def convoling_chunk(row, over_3_factor=0.1, chunk_duration=3, chunk_gen=2, only_slide=False):
    chunk_df = pd.DataFrame(columns=row.to_frame().T.columns)
    rows_to_add = []
    offset = row["OFFSET"]
    duration = row["DURATION"]
    
    #Ignore small duration (could be errors, play with this value)
    if(duration < 0.4):
        return chunk_df
    
    #print(row["IN FILE"], duration)
    if (duration <= chunk_duration and not only_slide):
        #Put the original bird call at...
        #1) Start of clip
        #print("start", row["IN FILE"])
        if (offset+chunk_duration) < row["CLIP LENGTH"]:
            #print("1")
            rows_to_add = create_chunk_row(row, rows_to_add, offset, chunk_duration)
            
        #2) End of clip
        if offset+duration-chunk_duration > 0: 
            #print("2")
            rows_to_add = create_chunk_row(row, rows_to_add, offset+duration-chunk_duration, chunk_duration)
            
        #3) Middle of clip
        if offset+duration-chunk_duration/2>0 and (offset+duration+chunk_duration/2)< row["CLIP LENGTH"]:
            #print("3")
            
            #Could be better placed in middle, maybe with some randomization?
            rows_to_add = create_chunk_row(row, rows_to_add, (offset+duration-chunk_duration/2), chunk_duration)
            
    
    #Longer than chunk duration
    else:
        #Perform Yan's Sliding Window operation
        clip_num=int(duration/(chunk_duration/2))
        for i in range(clip_num-1):
            new_start = offset+i*1.5
            new_end = offset + chunk_duration+i*1.5
            if ((offset+3)+i*1.5) < row["CLIP LENGTH"]:
                create_chunk_row(row, rows_to_add, new_start, chunk_duration) 
    
    #Add all new rows to our return df
    if (len(rows_to_add) == 0):
        return chunk_df
    
    chunk_df = pd.concat(rows_to_add,  ignore_index=True)

    return chunk_df

def dynamic_yan_chunking(df, chunk_count=5,chunk_duration=3, only_slide=False):
    return_df = pd.DataFrame(columns=df.columns)
    
    for index, row in df.iterrows():
        chunk_df = convoling_chunk(row, only_slide=only_slide)
        return_df = pd.concat([return_df, chunk_df],  ignore_index=True)
    
    return return_df


#dynamic_yan_chunking(test_anntotations, 3)    

In [5]:
test_anntotations

,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH
0,1.883721,0.116279,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC100027 - Southern Nightingale-Wren - Microce...,0,81.528163,44100,Microcerculus marginatus,0.543653,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
1,3.976744,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC100027 - Southern Nightingale-Wren - Microce...,0,81.528163,44100,Microcerculus marginatus,0.230806,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
2,5.976744,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC100027 - Southern Nightingale-Wren - Microce...,0,81.528163,44100,Microcerculus marginatus,0.204163,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
3,6.023256,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC100027 - Southern Nightingale-Wren - Microce...,0,81.528163,44100,Microcerculus marginatus,0.264257,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
4,6.069767,0.116279,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC100027 - Southern Nightingale-Wren - Microce...,0,81.528163,44100,Microcerculus marginatus,0.544028,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
...,...,...,...,...,...,...,...,...,...,...
55642,1.511628,0.325581,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC9881 - Blue-grey Tanager - Thraupis episcopu...,0,14.700000,44100,Thraupis episcopus,0.943661,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55643,1.860465,0.139535,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC9881 - Blue-grey Tanager - Thraupis episcopu...,0,14.700000,44100,Thraupis episcopus,0.855924,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55644,5.976744,0.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC9881 - Blue-grey Tanager - Thraupis episcopu...,0,14.700000,44100,Thraupis episcopus,0.775321,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55645,6.627907,0.255814,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,XC9881 - Blue-grey Tanager - Thraupis episcopu...,0,14.700000,44100,Thraupis episcopus,0.736337,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...


In [6]:
#V1


# def convoling_chunk(row, chunk_count=5, chunk_duration=3):
#     duration = row["DURATION"]
#     time_change = duration/chunk_duration/chunk_count
#     print(duration/chunk_duration/chunk_count)
#     print(row["OFFSET"], row["OFFSET"]+ row["DURATION"])
#     print("=================")
#     for i in range(chunk_count * 2):
#         print(row["OFFSET"] + (i-chunk_count) * time_change, row["OFFSET"] + (i-chunk_count) * time_change + chunk_duration)
#     print("========------------------------------=========")
#     return row.to_frame().T












#v2

# def convoling_chunk(row, over_3_factor=0.1, chunk_duration=3, chunk_gen=2):
#     chunk_df = pd.DataFrame(columns=row.to_frame().T.columns)
    
#     #Compute where sliding window starts and ends
#     start = min(row["OFFSET"] - over_3_factor * chunk_duration, row["OFFSET"] + row["DURATION"] - chunk_duration)
#     end = max(row["OFFSET"] + row["DURATION"]  + over_3_factor * chunk_duration, row["OFFSET"]  + chunk_duration)
    
#     #Generate each chunk in the window range
#     rows_to_add = []
#     time_change = (end-start) / chunk_gen
#     for i in range(chunk_gen):
#         chunk_row = row.copy()
#         chunk_row["DURATION"] = chunk_duration
#         chunk_row["OFFSET"] = start + i * time_change
#         rows_to_add.append(chunk_row.to_frame().T)
    
    
#     chunk_df = pd.concat(rows_to_add,  ignore_index=True)

#     return chunk_df

In [7]:
#annotation_chunker_no_duplicates(train_clips, 3, False)

In [8]:
bird_mixed = pd.read_csv("mixed_bird_binary_annotations_tweetynet_chunked.csv").head(30)

In [9]:
bird_mixed

,Unnamed: 0,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID
0,0,Aburria-aburri-257423.wav,24.633469,0,0.0,3.0,44100,bird
1,1,Aburria-aburri-257423.wav,24.633469,0,3.0,3.0,44100,bird
2,2,Aburria-aburri-257423.wav,24.633469,0,6.0,3.0,44100,bird
3,3,Aburria-aburri-257423.wav,24.633469,0,9.0,3.0,44100,bird
4,4,Aburria-aburri-257423.wav,24.633469,0,15.0,3.0,44100,bird
5,5,Aburria-aburri-257423.wav,24.633469,0,18.0,3.0,44100,bird
6,6,Aburria-aburri-257423.wav,24.633469,0,21.0,3.0,44100,bird
7,7,Aburria-aburri-387212.wav,84.480000,0,0.0,3.0,44100,bird
8,8,Aburria-aburri-387212.wav,84.480000,0,3.0,3.0,44100,bird
9,9,Aburria-aburri-387212.wav,84.480000,0,30.0,3.0,44100,bird


In [10]:
#NOTE ASSUMES ALL THE LABELS ARE THE SAME
def combine_chunked_df(df):
    return_df = pd.DataFrame(columns=df.columns)
    dfs = []
    for file in df["IN FILE"].unique():
        file_df = df[df["IN FILE"] == file]
        data_df = file_df.copy()

        file_df = file_df.sort_values(by="OFFSET")
        file_df["END"] = np.array(file_df["OFFSET"] + file_df["DURATION"])
        file_df["NEXT START"] = file_df["OFFSET"].shift(-1)
        file_df["LAST END"] = file_df["END"].shift(1)

        #IF TRUE, THEN THIS ROW IS SEPERATED ABOVE AND BELOW FROM ANY OTHER ANNOTATION
        temp = ~((file_df['END'] == file_df['NEXT START']) | (file_df['OFFSET'] == file_df['LAST END']))

        #SEE IF WE HAVE THE START OF A GROUPED ANNOTATION OR WE HAVE A INDEPETENT ANNOTATION
        file_df["GROUPS"] = ((temp.shift() != temp) | temp).cumsum()

        file_df = file_df[["OFFSET", "END", "NEXT START", "LAST END", "GROUPS"]]
        file_df = file_df.groupby("GROUPS").agg({"OFFSET": "min", "END": "max"}).reset_index(drop=True)
        
        file_df["DURATION"] = file_df["END"] - file_df["OFFSET"]
        file_df["IN FILE"] = file
        file_df["CLIP LENGTH"] = data_df.iloc[0]["CLIP LENGTH"]
        file_df["CHANNEL"] = data_df.iloc[0]["CHANNEL"]
        file_df["MANUAL ID"] = data_df.iloc[0]["MANUAL ID"]
        file_df["SAMPLE RATE"] = data_df.iloc[0]["SAMPLE RATE"]
        dfs.append(file_df)
        
    return pd.concat(dfs).reset_index(drop=True)
               
        
        
        
combine_chunked_df(bird_mixed)

,OFFSET,END,DURATION,IN FILE,CLIP LENGTH,CHANNEL,MANUAL ID,SAMPLE RATE
0,0.0,24.0,24.0,Aburria-aburri-257423.wav,24.633469,0,bird,44100
1,0.0,81.0,81.0,Aburria-aburri-387212.wav,84.480000,0,bird,44100
2,0.0,3.0,3.0,Accipiter-bicolor-451839.wav,5.668571,0,bird,44100
3,0.0,3.0,3.0,Accipiter-collaris-260335.wav,45.374694,0,bird,44100
4,6.0,9.0,3.0,Accipiter-collaris-260335.wav,45.374694,0,bird,44100
5,12.0,24.0,12.0,Accipiter-collaris-260335.wav,45.374694,0,bird,44100
6,27.0,30.0,3.0,Accipiter-collaris-260335.wav,45.374694,0,bird,44100
7,33.0,36.0,3.0,Accipiter-collaris-260335.wav,45.374694,0,bird,44100
8,39.0,45.0,6.0,Accipiter-collaris-260335.wav,45.374694,0,bird,44100
9,0.0,3.0,3.0,Accipiter-collaris-260336.wav,26.932245,0,bird,44100


In [11]:
bird_mixed["FOLDER"] =""
test = annotation_chunker_no_duplicates(bird_mixed, 3, True)
no_bird = combine_chunked_df(test[test["MANUAL ID"] == "no bird"])
no_bird

,OFFSET,END,DURATION,IN FILE,CLIP LENGTH,CHANNEL,MANUAL ID,SAMPLE RATE
0,12.0,15.0,3.0,Aburria-aburri-257423.wav,24.633469,0,no bird,44100
1,6.0,75.0,69.0,Aburria-aburri-387212.wav,84.480000,0,no bird,44100
2,81.0,84.0,3.0,Aburria-aburri-387212.wav,84.480000,0,no bird,44100
3,3.0,6.0,3.0,Accipiter-collaris-260335.wav,45.374694,0,no bird,44100
4,9.0,12.0,3.0,Accipiter-collaris-260335.wav,45.374694,0,no bird,44100
5,24.0,27.0,3.0,Accipiter-collaris-260335.wav,45.374694,0,no bird,44100
6,30.0,33.0,3.0,Accipiter-collaris-260335.wav,45.374694,0,no bird,44100
7,36.0,39.0,3.0,Accipiter-collaris-260335.wav,45.374694,0,no bird,44100
8,3.0,6.0,3.0,Accipiter-collaris-260336.wav,26.932245,0,no bird,44100
9,9.0,18.0,9.0,Accipiter-collaris-260336.wav,26.932245,0,no bird,44100


In [12]:
dynamic_yan_chunking(no_bird, 3, only_slide=True)

,OFFSET,END,DURATION,IN FILE,CLIP LENGTH,CHANNEL,MANUAL ID,SAMPLE RATE
0,12.0,15.0,3,Aburria-aburri-257423.wav,24.633469,0,no bird,44100
1,6.0,75.0,3,Aburria-aburri-387212.wav,84.48,0,no bird,44100
2,7.5,75.0,3,Aburria-aburri-387212.wav,84.48,0,no bird,44100
3,9.0,75.0,3,Aburria-aburri-387212.wav,84.48,0,no bird,44100
4,10.5,75.0,3,Aburria-aburri-387212.wav,84.48,0,no bird,44100
...,...,...,...,...,...,...,...,...
128,111.0,120.0,3,Accipiter-poliogaster-121768.wav,121.751995,0,no bird,44100
129,112.5,120.0,3,Accipiter-poliogaster-121768.wav,121.751995,0,no bird,44100
130,114.0,120.0,3,Accipiter-poliogaster-121768.wav,121.751995,0,no bird,44100
131,115.5,120.0,3,Accipiter-poliogaster-121768.wav,121.751995,0,no bird,44100


In [13]:
test[test["MANUAL ID"] == "no bird"].shape

(70, 9)